Deep learning with PyTorch Lightning

PyTorch Lightning makes the deep learning workflow even simpler by encapsulating more procedures into the module. Compared to pytorch, it also includes training loop, validation loop and configuration of optimizers.

The PyTorch Lightning module hook includes:
- Computational layer (model architecture)
- Forward 
- Configure optimizers
- Training step
- Validation step

Link to the tutorial: <br>
https://lightning.ai/docs/pytorch/latest/starter/introduction.html

In [9]:
import os
import torch
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L

In [3]:
# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

In [4]:
# define the LightningModule
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [5]:
# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

In [6]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:02<00:00, 4078336.79it/s]


Extracting /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw/train-images-idx3-ubyte.gz to /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 44914977.32it/s]


Extracting /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5450403.31it/s]


Extracting /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 8554346.10it/s]

Extracting /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/yangliu/dryang/MLpilot/framework/torch/MNIST/raw



In [7]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/yangliu/venv/ml/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: /Users/yangliu/dryang/MLpilot/framework/torch/lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-train

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


The Lightning Trainer automates 40+ tricks including:

- Epoch and batch iteration
- optimizer.step(), loss.backward(), optimizer.zero_grad() calls
- Calling of model.eval(), enabling/disabling grads during evaluation
- Checkpoint Saving and Loading
- Tensorboard (see loggers options)
- Multi-GPU support
- TPU
- 16-bit precision AMP support

In [10]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = torch.rand(4, 28 * 28, device=autoencoder.device)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[-1.3215,  1.0051, -0.7035],
        [-1.4744,  1.0465, -0.6635],
        [-1.5270,  1.1226, -0.6501],
        [-1.5873,  1.1389, -0.7155]], device='mps:0',
       grad_fn=<LinearBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
